In [1]:
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Scatter_Plot.ipynb
%run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/QQ_Plot.ipynb
# %run D:/UTD/UTDFall2023/Calibration-of-LoRaNodes-using-Super-Learners/plots/Feature_Importance_Plot_Random_Forest_Regressor.ipynb

In [2]:
# def LGBM_Regression(X_train,X_test,y_train,y_test,filtered_data):

#     def objective(params):
#         n_estimators, max_depth, learning_rate, num_leaves, min_child_samples = params

#         model = LGBMRegressor(
#             n_estimators=n_estimators,
#             max_depth=max_depth,
#             learning_rate=learning_rate,
#             num_leaves=num_leaves,
#             min_child_samples=min_child_samples,
#             random_state=1
#         )

#         model.fit(X_train, y_train)
#         predict_train = model.predict(X_train)
#         r2 = r2_score(y_train, predict_train)

#         return -r2  # Negate the r2 score for minimization

#     # Define the hyperparameter space
#     param_space = [
#         Integer(50, 300, name="n_estimators"),
#         Integer(3, 30, name="max_depth"),
#         Real(0.01, 0.3, name="learning_rate"),
#         Integer(20, 150, name="num_leaves"),
#         Integer(20, 100, name="min_child_samples")
#     ]

#     # Perform Bayesian optimization
#     results = gp_minimize(objective, param_space, n_calls=50, random_state=0)

#     # Extract the best hyperparameters
#     n_estimators, max_depth, learning_rate, num_leaves, min_child_samples = results.x

#     # Create and train the model with the best hyperparameters
#     best_model = LGBMRegressor(
#         n_estimators=n_estimators,
#         max_depth=max_depth,
#         learning_rate=learning_rate,
#         num_leaves=num_leaves,
#         min_child_samples=min_child_samples,
#         random_state=1
#     )
#     best_model.fit(X_train, y_train)
#     predict_train = best_model.predict(X_train)
#     predict_test = best_model.predict(X_test)

#     train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
#     test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

#     # Concatenate the DataFrames
#     combined_df = pd.concat([train_df, test_df])
#     combined_df = combined_df.sort_index()
#     # Print or use the combined DataFrame as needed
#     print(combined_df)
    
#     r2_score_train = round(metrics.r2_score(y_train, predict_train),2) 
#     r2_score_test = round(metrics.r2_score(y_test, predict_test),2)
    
#     if (v == 'dCn'):
#         unit = 'dCn'
#     else:
#         unit = 'pm_conc'
        
#     Scatter_Plot(combined_df, train_df, test_df, r2_score_train, r2_score_test, v, unit)
#     qq_plot(test_df, v, unit)
#     #Feature_Importance_Random_Forest_Regressor(best_model, filtered_data, v, unit)
    
#     return best_model,results.x, r2_score_test

In [ ]:

def LGBM_Regression(X_train, X_test, y_train, y_test, filtered_data):
    # Function to be optimized by Bayesian Optimization
    def lgbm_regressor_cv(n_estimators, max_depth, learning_rate, num_leaves, min_child_samples):
        # Ensure parameters are in proper format
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        num_leaves = int(num_leaves)
        min_child_samples = int(min_child_samples)
        
        # Define and fit the model
        model = LGBMRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            num_leaves=num_leaves,
            min_child_samples=min_child_samples,
            random_state=1
        )
        model.fit(X_train, y_train)

        # Predict and calculate R2 score
        predict_test = model.predict(X_test)
        r2_score_test = metrics.r2_score(y_test, predict_test)
        return r2_score_test

    # Setting ranges for hyperparameters
    pbounds = {
        'n_estimators': (50, 500),
        'max_depth': (5, 30),
        'learning_rate': (0.01, 0.3),
        'num_leaves': (20, 50),
        'min_child_samples': (10, 30)
    }

    # Bayesian Optimization
    optimizer = BayesianOptimization(
        f=lgbm_regressor_cv,
        pbounds=pbounds,
        random_state=1
    )
    optimizer.maximize(init_points=5, n_iter=15)

    # Optimal parameters
    params = optimizer.max['params']
    
    params['n_estimators'] = int(params['n_estimators'])
    params['max_depth'] = int(params['max_depth'])
    params['learning_rate'] = np.round(params['learning_rate'],4)
    params['num_leaves'] = int(params['num_leaves'])
    params['min_child_samples'] = int(params['min_child_samples'])
    
    print("Optimal Parameters:", params)

    # Create and train model with optimal parameters
    best_model = LGBMRegressor(
        n_estimators = params['n_estimators'],
        max_depth = params['max_depth'],
        learning_rate = params['learning_rate'],
        num_leaves = params['num_leaves'],
        min_child_samples = params['min_child_samples'],
        random_state=1
    )
    best_model.fit(X_train, y_train)

    # Predictions
    predict_train = best_model.predict(X_train)
    predict_test = best_model.predict(X_test)

    # DataFrame Creation
    train_df = pd.DataFrame({'Actual': y_train, 'Predicted': predict_train, 'Category': 'Training'}, index=y_train.index)
    test_df = pd.DataFrame({'Actual': y_test, 'Predicted': predict_test, 'Category': 'Testing'}, index=y_test.index)

    # Concatenating and Sorting
    combined_df = pd.concat([train_df, test_df])
    combined_df = combined_df.sort_index()

    # R2 Score
    r2_score_train = round(metrics.r2_score(y_train, predict_train), 2)
    r2_score_test = round(metrics.r2_score(y_test, predict_test), 2)
    print('R2 Score Train:', r2_score_train)
    print('R2 Score Test:', r2_score_test)

    return best_model,params,r2_score_test